In [1]:
!which python

/Users/apabook/Desktop/ds-deployment/.venv/bin/python


In [9]:

import json
import pprint
import uuid

import requests



# Testing LL Api for PULL transfer

---

### Consts

In [3]:
data_space_provider = "http://127.0.0.1:1234"
data_space_consumer = "http://127.0.0.1:1235"
context_broker = "http://fiware-orion:1026"
streaming_processor = "http://streaming-testing-service:1237"

catalog_id = ""
dataservice_id = ""
agreement_id = ""
agreement_pid = ""

### Setup Catalog, Dataservice and agreements

In [4]:
payload = {
    "foaf:homepage": "My catalog in Dataspace provider",
    "dct:title": "My catalog in Dataspace provider"
}
headers = {
    "Content-Type": "application/json",
}
url = data_space_provider + "/api/v1/catalogs"
response = requests.request("POST", url, headers=headers, data=json.dumps(payload))
response_as_json = response.json()
catalog_id = response_as_json["@id"]

pprint.pprint(response.json())

{'@context': 'https://w3id.org/dspace/2024/1/context.json',
 '@id': 'fdcc279d-0286-4abb-bf5f-2b2509172578',
 '@type': 'dcat:Catalog',
 'dcat:dataset': [],
 'dcat:keyword': '',
 'dcat:service': [],
 'dcat:theme': '',
 'dct:conformsTo': None,
 'dct:creator': None,
 'dct:description': [],
 'dct:identifier': 'fdcc279d-0286-4abb-bf5f-2b2509172578',
 'dct:issued': '2024-12-19T13:31:47.236411',
 'dct:modified': None,
 'dct:title': 'My catalog in Dataspace provider',
 'dspace:extraFields': None,
 'dspace:participantId': None,
 'foaf:homepage': 'My catalog in Dataspace provider',
 'odrl:hasPolicy': None}


In [5]:
payload = {
    "dcat:endpointURL": context_broker + "/v2/entities"
}
headers = {
    "Content-Type": "application/json",
}
url = data_space_provider + "/api/v1/catalogs/" + catalog_id + "/data-services"
response = requests.request("POST", url, headers=headers, data=json.dumps(payload))
response_as_json = response.json()
dataservice_id = response_as_json["@id"]

pprint.pprint(response.json())

{'@context': 'https://w3id.org/dspace/2024/1/context.json',
 '@id': 'a8a5a215-2d04-4158-b5ce-a080cabef9ee',
 '@type': 'dcat:DataService',
 'dcat:endpointDescription': '',
 'dcat:endpointURL': 'http://fiware-orion:1026/v2/entities',
 'dcat:keyword': '',
 'dcat:theme': '',
 'dct:conformsTo': None,
 'dct:creator': None,
 'dct:description': [],
 'dct:identifier': 'a8a5a215-2d04-4158-b5ce-a080cabef9ee',
 'dct:issued': '2024-12-19T13:31:47.578344',
 'dct:modified': None,
 'dct:title': None,
 'dspace:extraFields': None,
 'odrl:hasPolicy': None}


In [6]:
payload = {
    "dataServiceId": dataservice_id
}
headers = {
    "Content-Type": "application/json",
}
url = data_space_provider + "/api/v1/agreements"
response = requests.request("POST", url, headers=headers, data=json.dumps(payload))
response_as_json = response.json()
agreement_id = response_as_json["agreement_id"]
agreement_pid = "urn:uuid:" + agreement_id

pprint.pprint(response.json())

{'agreement_id': 'b0d5fdc4-bd13-4dcc-9dbb-88079ce7fce5',
 'data_service_id': 'a8a5a215-2d04-4158-b5ce-a080cabef9ee',
 'identity': None,
 'identity_token': None}


### Fake Callback server
In the contiguous file called `pull_testing_no_consumer_callback_server` there is a script \
to deploy a fake server to capture de callback messages for the Transfer Process Protocol.

## Transfer Process Protocol

### Transfer Request and Transfer Start

In [51]:
transfer_message = {
    "@type": "dspace:TransferRequestMessage",
    "@context": "https://w3id.org/dspace/2024/1/context.json",
    "dct:format": "http+pull",
    "dspace:agreementId": agreement_pid,
    "dspace:consumerPid": "urn:uuid:" + str(uuid.uuid4()),
    "dspace:callbackAddress": "http://localhost:5000"
}

url = data_space_provider + "/transfers/request"
headers = {
    "Content-Type": "application/json",
}
response = requests.request("POST", url, headers=headers, data=json.dumps(transfer_message))
provider_pid = response.json().get("dspace:providerPid").replace("urn:uuid:", "")
consumer_pid = response.json().get("dspace:consumerPid").replace("urn:uuid:", "")
response.json()

{'@context': 'https://w3id.org/dspace/2024/1/context.json',
 '@type': 'dspace:TransferProcess',
 'dspace:providerPid': 'urn:uuid:5b41985b-00d4-42b5-bf91-7471171ff9dd',
 'dspace:consumerPid': 'urn:uuid:ab2fedaf-ea5f-4782-8e42-2af5f327f7d8',
 'dspace:state': 'dspace:REQUESTED'}

### Transfer Current Status

In [52]:
url = data_space_provider + "/transfers/" + provider_pid
response = requests.request("GET", url)
response.json()

{'@context': 'https://w3id.org/dspace/2024/1/context.json',
 '@type': 'dspace:TransferProcess',
 'dspace:providerPid': 'urn:uuid:5b41985b-00d4-42b5-bf91-7471171ff9dd',
 'dspace:consumerPid': 'urn:uuid:ab2fedaf-ea5f-4782-8e42-2af5f327f7d8',
 'dspace:state': 'dspace:STARTED'}

### Transfer Suspend


In [53]:
transfer_message = {
    "@type": "dspace:TransferSuspensionMessage",
    "@context": "https://w3id.org/dspace/2024/1/context.json",
    "dspace:providerPid": "urn:uuid:" + provider_pid,
    "dspace:consumerPid": "urn:uuid:" + consumer_pid,
    "dspace:code": "200",
    "dspace:reason": [
        "blabla"
    ]
}

url = data_space_provider + "/transfers/suspension"
headers = {
    "Content-Type": "application/json",
}
response = requests.request("POST", url, headers=headers, data=json.dumps(transfer_message))
response.json()

{'@context': 'https://w3id.org/dspace/2024/1/context.json',
 '@type': 'dspace:TransferProcess',
 'dspace:providerPid': 'urn:uuid:5b41985b-00d4-42b5-bf91-7471171ff9dd',
 'dspace:consumerPid': 'urn:uuid:ab2fedaf-ea5f-4782-8e42-2af5f327f7d8',
 'dspace:state': 'dspace:SUSPENDED'}

### Transfer Complete


In [43]:
transfer_message = {
    "@type": "dspace:TransferCompletionMessage",
    "@context": "https://w3id.org/dspace/2024/1/context.json",
    "dspace:providerPid": "urn:uuid:" + provider_pid,
    "dspace:consumerPid": "urn:uuid:" + consumer_pid,
}

url = data_space_provider + "/transfers/completion"
headers = {
    "Content-Type": "application/json",
}
response = requests.request("POST", url, headers=headers, data=json.dumps(transfer_message))
response.json()

{'@context': 'https://w3id.org/dspace/2024/1/context.json',
 '@type': 'dspace:TransferProcess',
 'dspace:providerPid': 'urn:uuid:4a24add9-55d6-4f58-8168-68235b72bdf7',
 'dspace:consumerPid': 'urn:uuid:c7521b31-d797-46c4-831d-82464a61d521',
 'dspace:state': 'dspace:COMPLETED'}

### Transfer Termination


In [44]:
transfer_message = {
    "@type": "dspace:TransferTerminationMessage",
    "@context": "https://w3id.org/dspace/2024/1/context.json",
    "dspace:providerPid": "urn:uuid:" + provider_pid,
    "dspace:consumerPid": "urn:uuid:" + consumer_pid,
    "dspace:code": "200",
    "dspace:reason": [
        "blabla"
    ]
}

url = data_space_provider + "/transfers/termination"
headers = {
    "Content-Type": "application/json",
}
response = requests.request("POST", url, headers=headers, data=json.dumps(transfer_message))
response.json()

{'@context': 'https://w3id.org/dspace/2024/1/context.json',
 '@type': 'dspace:TransferError',
 'dspace:providerPid': '123',
 'dspace:consumerPid': '123',
 'dspace:code': 'TRANSFER_ERROR_CODE',
 'dspace:reason': ['Protocol Error. dspace:TransferRequestMessage is not allowed here. Current state is dspace:COMPLETED']}